<a href="https://colab.research.google.com/github/AquaSense-Capstone/Baseline_Model-ML/blob/main/Capstone%20Project/Model_v2/Model_new_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://github.com/AquaSense-Capstone/Baseline_Model-ML/raw/main/Capstone%20Project/Model_v2/Dataset/Tinggi_Air.csv
!wget https://github.com/AquaSense-Capstone/Baseline_Model-ML/raw/main/Capstone%20Project/Model_v2/Dataset/laporan_iklim_harian.csv

In [26]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import resample
from keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow import lite

In [2]:
# Membaca dataset tinggi_air
df_tinggi_air = pd.read_csv('Tinggi_Air.csv', parse_dates=['tanggal'])

# Membaca dataset laporan_iklim_harian
df_laporan_iklim = pd.read_csv('laporan_iklim_harian.csv', parse_dates=['Tanggal'], dayfirst=True)

# Mengganti tanda koma dengan titik pada kolom 'curah_hujan'
df_laporan_iklim['curah_hujan'] = df_laporan_iklim['curah_hujan'].str.replace(',', '.')
# Mengonversi kolom 'curah_hujan' ke tipe data float
df_laporan_iklim['curah_hujan'] = pd.to_numeric(df_laporan_iklim['curah_hujan'], errors='coerce')

# Menggabungkan dataframe berdasarkan tanggal
result_df = pd.merge(df_tinggi_air, df_laporan_iklim, left_on=df_tinggi_air['tanggal'].dt.date, right_on=df_laporan_iklim['Tanggal'].dt.date, how='left')

dataset = result_df[['nama_pintu_air', 'lokasi', 'tinggi_normal' , 'tinggi_siaga_3' , 'tinggi_siaga_2' ,'tanggal', 'tinggi_air', 'curah_hujan', 'status_siaga']]
# Menampilkan hasil
print(dataset)

             nama_pintu_air              lokasi  tinggi_normal  \
0            PS. Angke Hulu               Angke           1490   
1            PS. Angke Hulu               Angke           1490   
2            PS. Angke Hulu               Angke           1490   
3      PS. Katulampa (Hulu)            Ciliwung            790   
4                 PS. Depok            Ciliwung           1990   
...                     ...                 ...            ...   
13303         PA. Manggarai            Ciliwung           7490   
13304            P.A. Karet  Banjir Kanal Barat           4490   
13305        PS. Angke Hulu               Angke           1490   
13306   P.A. Jembatan Merah            Ciliwung           1490   
13307    P.A. Ancol Flusing            Ciliwung           1790   

       tinggi_siaga_3  tinggi_siaga_2             tanggal  tinggi_air  \
0                2490            2990 2019-12-31 23:00:00         900   
1                2490            2990 2019-12-31 23:00:00    

In [3]:
# Menghitung jumlah baris dengan nilai NaN dalam DataFrame
jumlah_nan = dataset.isna().sum()

print("Jumlah NaN untuk setiap kolom:")
print(jumlah_nan)

Jumlah NaN untuk setiap kolom:
nama_pintu_air       0
lokasi               0
tinggi_normal        0
tinggi_siaga_3       0
tinggi_siaga_2       0
tanggal              0
tinggi_air           0
curah_hujan       2280
status_siaga         0
dtype: int64


In [4]:
dataset = pd.DataFrame(dataset)
# Mengganti nilai '8888' dan '9999' dengan NaN
dataset['curah_hujan']=dataset['curah_hujan'].replace([8888, 9999], pd.NA)

# Menghapus baris yang masih memiliki NaN
dataset.dropna(inplace=True)

# # Menghapus duplikat data
dataset = dataset.drop_duplicates()

jumlah_nan = dataset.isna().sum()
print("Jumlah NaN untuk setiap kolom:")
print(jumlah_nan)

Jumlah NaN untuk setiap kolom:
nama_pintu_air    0
lokasi            0
tinggi_normal     0
tinggi_siaga_3    0
tinggi_siaga_2    0
tanggal           0
tinggi_air        0
curah_hujan       0
status_siaga      0
dtype: int64


In [5]:
dataset

,nama_pintu_air,lokasi,tinggi_normal,tinggi_siaga_3,tinggi_siaga_2,tanggal,tinggi_air,curah_hujan,status_siaga
0,PS. Angke Hulu,Angke,1490,2490,2990,2019-12-31 23:00:00,900,0.0,Normal
3,PS. Katulampa (Hulu),Ciliwung,790,1490,1990,2019-12-31 23:40:00,480,0.0,Normal
4,PS. Depok,Ciliwung,1990,2690,3490,2019-12-31 23:40:00,1720,0.0,Normal
5,PA. Manggarai,Ciliwung,7490,8490,9490,2019-12-31 23:40:00,6320,0.0,Normal
6,P.A. Karet,Banjir Kanal Barat,4490,5490,5990,2019-12-31 23:40:00,3290,0.0,Normal
...,...,...,...,...,...,...,...,...,...
12880,P.A. Karet,Banjir Kanal Barat,4490,5490,5990,2020-01-30 23:40:00,2710,22.0,Normal
12881,PS. Angke Hulu,Angke,1490,2490,2990,2020-01-30 23:40:00,850,22.0,Normal
12882,P.A. Istiqlal,Ciliwung,2490,2990,3490,2020-01-30 23:40:00,2250,22.0,Normal
12883,P.A. Jembatan Merah,Ciliwung,1490,1990,2490,2020-01-30 23:40:00,1760,22.0,Siaga 2


In [6]:
# ubah status siaga yang tidak sesuai
for index, row in dataset.iterrows():
    if row['tinggi_air'] <= row['tinggi_normal']:
        dataset.at[index, 'status_siaga'] = 'Normal'
    elif row['tinggi_air'] > row['tinggi_normal'] and row['tinggi_air'] <= row['tinggi_siaga_3']:
        dataset.at[index, 'status_siaga'] = 'Siaga 3'
    elif row['tinggi_air'] > row['tinggi_siaga_3'] and row['tinggi_air'] <= row['tinggi_siaga_2']:
        dataset.at[index, 'status_siaga'] = 'Siaga 2'
    else:
        dataset.at[index, 'status_siaga'] = 'Siaga 1'

print(dataset['status_siaga'].value_counts())

status_siaga
Normal     6416
Siaga 3    1367
Siaga 2     649
Siaga 1     197
Name: count, dtype: int64


In [7]:
# oversampling
# List untuk menyimpan data oversampled
oversampled_data = []

# Oversampling berbasis treshold
for status_siaga in dataset['status_siaga'].unique():
    # Filter data berdasarkan status_siaga
    subset_data = dataset[dataset['status_siaga'] == status_siaga]
    
    # Ambil treshold dari kolom yang sesuai dengan status_siaga
    treshold_col = f'tinggi_{status_siaga.lower().replace(" ", "_")}'
    
    # Hitung jumlah sampel yang dibutuhkan untuk oversampling
    target_samples = dataset['status_siaga'].value_counts()['Normal'] 
    current_samples = len(subset_data)
    oversample_factor = max(1, target_samples // current_samples)
    
    # Lakukan oversampling
    oversampled_subset = resample(subset_data, replace=True, n_samples=oversample_factor * current_samples)
    
    # Tambahkan hasil oversampling ke list
    oversampled_data.append(oversampled_subset)

# Gabungkan dan acak data oversampled
df_oversampled = pd.concat(oversampled_data)
df_oversampled = df_oversampled.sample(frac=1).reset_index(drop=True)  # Acak baris

# Cetak hasil oversampling
print(df_oversampled['status_siaga'].value_counts())

status_siaga
Normal     6416
Siaga 1    6304
Siaga 2    5841
Siaga 3    5468
Name: count, dtype: int64


In [8]:
dataset = df_oversampled
dataset

,nama_pintu_air,lokasi,tinggi_normal,tinggi_siaga_3,tinggi_siaga_2,tanggal,tinggi_air,curah_hujan,status_siaga
0,P.A. Ancol Flusing,Ciliwung,1790,1890,2190,2020-01-28 17:40:00,1740,4.2,Normal
1,PA. Manggarai,Ciliwung,7490,8490,9490,2020-01-08 13:10:00,6290,2.8,Normal
2,P.A. Ancol Flusing,Ciliwung,1790,1890,2190,2020-01-19 23:10:00,1630,3.8,Normal
3,P.A. Ancol Flusing,Ciliwung,1790,1890,2190,2020-01-01 07:10:00,2210,145.3,Siaga 1
4,PS. Katulampa (Hulu),Ciliwung,790,1490,1990,2020-01-26 16:00:00,110,7.4,Normal
...,...,...,...,...,...,...,...,...,...
24024,P.A. Jembatan Merah,Ciliwung,1490,1990,2490,2020-01-01 13:20:00,2700,145.3,Siaga 1
24025,P.A. Istiqlal,Ciliwung,2490,2990,3490,2020-01-21 01:10:00,2520,2.4,Siaga 3
24026,P.A. Jembatan Merah,Ciliwung,1490,1990,2490,2020-01-18 12:10:00,2030,78.5,Siaga 2
24027,P.A. Jembatan Merah,Ciliwung,1490,1990,2490,2020-01-24 12:40:00,2660,7.8,Siaga 1


In [9]:
features = dataset[['tinggi_normal','tinggi_siaga_3','tinggi_siaga_2','tinggi_air','curah_hujan']].values
target = dataset['status_siaga'].values

# Normalisasi fitur
scaler = MinMaxScaler()
features_normalized = scaler.fit_transform(features)

# Split data menjadi data pelatihan dan pengujian
X_train, X_test, y_train, y_test = train_test_split(features_normalized, target, test_size=0.2, random_state=42)

# Reshape data untuk sesuai dengan input model RNN
X_train_reshaped = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test_reshaped = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

In [10]:
# Konversi label-string ke label-integer
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Konversi label-integer ke one-hot encoding
y_train_onehot = to_categorical(y_train_encoded)
y_test_onehot = to_categorical(y_test_encoded)

In [11]:
# Build the RNN model using TensorFlow
model = Sequential([
    LSTM(units=50, activation='relu', input_shape=(1, X_train.shape[1]), return_sequences=True),
    Dropout(0.2),
    LSTM(units=50, activation='relu', return_sequences=True),
    LSTM(units=50, activation='relu'),
    Dense(units=4, activation='softmax')
])

# Compile model dengan optimizer Adam dan loss categorical_crossentropy
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Training model
model.fit(X_train_reshaped, y_train_onehot, epochs=50, batch_size=32, verbose=2)

# Evaluasi model
y_pred = model.predict(X_test_reshaped)
y_pred_classes = np.argmax(y_pred, axis=1)
accuracy = accuracy_score(y_test_encoded, y_pred_classes)
print(f'Accuracy: {accuracy}')

Epoch 1/50
601/601 - 9s - loss: 1.0019 - accuracy: 0.5515 - 9s/epoch - 15ms/step
Epoch 2/50
601/601 - 3s - loss: 0.4084 - accuracy: 0.8423 - 3s/epoch - 4ms/step
Epoch 3/50
601/601 - 3s - loss: 0.2940 - accuracy: 0.8860 - 3s/epoch - 4ms/step
Epoch 4/50
601/601 - 3s - loss: 0.2483 - accuracy: 0.9053 - 3s/epoch - 4ms/step
Epoch 5/50
601/601 - 3s - loss: 0.2253 - accuracy: 0.9120 - 3s/epoch - 4ms/step
Epoch 6/50
601/601 - 3s - loss: 0.2038 - accuracy: 0.9221 - 3s/epoch - 4ms/step
Epoch 7/50
601/601 - 3s - loss: 0.1853 - accuracy: 0.9290 - 3s/epoch - 4ms/step
Epoch 8/50
601/601 - 3s - loss: 0.1697 - accuracy: 0.9360 - 3s/epoch - 4ms/step
Epoch 9/50
601/601 - 3s - loss: 0.1613 - accuracy: 0.9379 - 3s/epoch - 4ms/step
Epoch 10/50
601/601 - 3s - loss: 0.1434 - accuracy: 0.9458 - 3s/epoch - 4ms/step
Epoch 11/50
601/601 - 3s - loss: 0.1413 - accuracy: 0.9467 - 3s/epoch - 4ms/step
Epoch 12/50
601/601 - 3s - loss: 0.1290 - accuracy: 0.9502 - 3s/epoch - 5ms/step
Epoch 13/50
601/601 - 3s - loss: 0.1

In [25]:
tinggi_normal = 1200 # batas tinggi air normal
siaga3 = 1900 # batas siaga 3
siaga2 = 2700 # batas siaga 2
tinggi_air = 2700 # tinggi air saat ini
curah_hujan = 100

# Normalisasi data asli dengan skalar yang telah digunakan
new_data_raw = np.array([[tinggi_normal, siaga3, siaga2, tinggi_air, curah_hujan]])  # inputan: tinggi_normal','tinggi_siaga_3','tinggi_siaga_2','tinggi_air','curah_hujan'
new_data_normalized = scaler.transform(new_data_raw)

# Reshape data untuk sesuai dengan input model RNN
new_data_normalized_reshaped = np.reshape(new_data_normalized, (new_data_normalized.shape[0], 1, new_data_normalized.shape[1]))

# Prediksi menggunakan model
new_data_pred_probs = model.predict(new_data_normalized_reshaped)
new_data_pred_classes = np.argmax(new_data_pred_probs, axis=1)

# Mendekode hasil prediksi ke dalam kategori status siaga
new_data_pred_decoded = label_encoder.inverse_transform(new_data_pred_classes)

# Cetak hasil prediksi dalam format label encoded dan decoded
print(f'Predicted Status Siaga (Encoded): {new_data_pred_classes}')
print(f'Predicted Status Siaga (Decoded): {new_data_pred_decoded}')

# Menampilkan klasifikasi status siaga
classification_result = dict(zip(label_encoder.classes_, new_data_pred_probs.flatten()))
print("Classification Result:")
for status, probability in classification_result.items():
    print(f'{status}: {probability * 100:.2f}%')

1/1 [==============================] - 0s 56ms/step
Predicted Status Siaga (Encoded): [1]
Predicted Status Siaga (Decoded): ['Siaga 1']
Classification Result:
Normal: 0.00%
Siaga 1: 77.37%
Siaga 2: 22.63%
Siaga 3: 0.00%


In [27]:
# Simpan model Keras ke dalam file
model.save('model_keras.h5')

# Muat model Keras dari file
loaded_model = tf.keras.models.load_model('model_keras.h5')

c:\Users\ThinkPad\AppData\Local\Programs\Python\Python38\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [28]:
converter = lite.TFLiteConverter.from_keras_model(loaded_model)

converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.experimental_new_converter=True
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS,
tf.lite.OpsSet.SELECT_TF_OPS]

tfmodel = converter.convert()
open('model.tflite', 'wb').write(tfmodel)

INFO:tensorflow:Assets written to: C:\Users\ThinkPad\AppData\Local\Temp\tmpwy6z4sj7\assets


INFO:tensorflow:Assets written to: C:\Users\ThinkPad\AppData\Local\Temp\tmpwy6z4sj7\assets


82496